## Transform Payments Data
1. Extract Date and Time from payment_timestamp and create new columns payment_date and payment_time
1. Map payment_status to contain descriptive values  
   (1-Success, 2-Pending, 3-Cancelled, 4-Failed)
1. Write transformed data to the Silver schema 

In [0]:
use catalog pavan_catalog_all;
use schema bronze;

In [0]:
%python
# Ingest raw data into Bronze
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

schema = StructType([
    StructField("payment_id", StringType(), True),
    StructField("order_id", StringType(), True),
    StructField("payment_timestamp", TimestampType(), True),
    StructField("payment_status", IntegerType(), True),
    StructField("payment_method", StringType(), True)
])

raw_df = (
    spark.read
    .option("delimiter", ",")
    .option("header", "false")
    .schema(schema)
    .csv("abfss://landing@parkaru15sa.dfs.core.windows.net/payments/")
)
display(raw_df)

In [0]:
%python
raw_df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("pavan_catalog_all.bronze.bronze_payments")

In [0]:
create or replace view 
  pavan_catalog_all.bronze.v_payments
AS
SELECT * from bronze.bronze_payments

In [0]:
SELECT payment_id,
       order_id,
       payment_timestamp,
       payment_status,
       payment_method
  FROM pavan_catalog_all.bronze.bronze_payments;

### 1. Extract Date and Time from payment_timestamp
>  [Documentation for date_format function](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/date_format)

In [0]:
SELECT payment_id,
       order_id,
       CAST(date_format(payment_timestamp,'yyyy-MM-dd') AS DATE) AS payment_date,
       date_format(payment_timestamp,'HH:mm:ss') AS payment_time,
       payment_status,
       payment_method
  FROM pavan_catalog_all.bronze.bronze_payments;

### 2. Map payment_status to contain descriptive values  
   (1-Success, 2-Pending, 3-Cancelled, 4-Failed)

In [0]:
SELECT payment_id,
       order_id,
       CAST(date_format(payment_timestamp,'yyyy-MM-dd') AS DATE) AS payment_date,
       date_format(payment_timestamp,'HH:mm:ss') AS payment_time,
       CASE payment_status
         WHEN 1 THEN 'Success'
         WHEN 2 THEN 'Pending'
         WHEN 3 THEN 'Cancelled'
         WHEN 4 THEN 'Failed'
       END AS payment_status,  
       payment_method
  FROM pavan_catalog_all.bronze.bronze_payments;

### 3. Write transformed data to the Silver schema 

In [0]:
CREATE TABLE pavan_catalog_all.silver.payments
AS
SELECT payment_id,
       order_id,
       CAST(date_format(payment_timestamp,'yyyy-MM-dd') AS DATE) AS payment_date,
       date_format(payment_timestamp,'HH:mm:ss') AS payment_time,
       CASE payment_status
         WHEN 1 THEN 'Success'
         WHEN 2 THEN 'Pending'
         WHEN 3 THEN 'Cancelled'
         WHEN 4 THEN 'Failed'
       END AS payment_status,  
       payment_method
  FROM pavan_catalog_all.bronze.bronze_payments;

In [0]:
SELECT * FROM pavan_catalog_all.silver.payments;